In [48]:
import pandas as pd
import re
from catboost import CatBoostRegressor

In [49]:
colunas = ['Semana','Empresa','Setor do LinkedIn','Número de Funcionários','Cargo','STATUS','Portfólio', 'Hunter']

In [50]:
treino = pd.read_csv('PAC - Contatos NProd.csv', usecols=colunas)

In [ ]:
teste = pd.read_csv('C:\Users\gabri\Documents\PROJETOS\PY\PJ\Comercial\PAC\apollo-contacts-export(1).xlsx', header=None, names=colunas).drop('STATUS', axis=1)

In [51]:
dates_before_50275 = treino.loc[:50274, 'Semana'].str.extract(r'(\d{2}/\d{2}/\d{4})')[0]
dates_before_50275 = pd.to_datetime(dates_before_50275, format='mixed', dayfirst=0).dt.strftime('%d/%m/%y')

treino.loc[:50274, 'Semana'] = dates_before_50275

dates_after_50275 = treino.loc[50275:, 'Semana'].str.extract(r'(\d{2}/\d{2}/\d{4})')[0]
dates_after_50275 = pd.to_datetime(dates_after_50275, format='mixed', dayfirst=1).dt.strftime('%d/%m/%y')

treino.loc[50275:, 'Semana'] = dates_after_50275

In [52]:
def adjust_num_func(row):
    if row['Hunter'] == 'gabrial.agra' and pd.to_numeric(row['Número de Funcionários'], errors='coerce') < 70:
        return pd.to_numeric(row['Número de Funcionários'], errors='coerce') * 1000
    return row['Número de Funcionários']

treino['Número de Funcionários'] = treino.apply(adjust_num_func, axis=1)

In [53]:
treino = treino.dropna(subset=['Hunter'])

In [54]:
job_title_replacements = [
    ('ceo', 'CEO'),
    ('chief executive officer', 'CEO'),
    ('cdo', 'CDO'),
    ('chief data officer', 'CDO'),
    ('cto', 'CTO'),
    ('chief technology officer', 'CTO'),
    ('cio', 'CIO'),    
    ('chief information officer', 'CIO'),
    ('coo', 'COO'),
    ('chief operating officer', 'COO'),
    ('cfo', 'CFO'),
    ('chief financial officer', 'CFO'),
    ('cmo', 'CMO'),
    ('chief marketing officer', 'CMO'), 
    ('cpo', 'CPO'),
    ('chief product officer', 'CPO'),
    ('cco', 'CCO'),
    ('chief compliance officer', 'CCO'),
    ('chief', 'Outro C-Level'),
    ('Presidente', 'Presidente/Vice Presidente'),
    ('vice-presidente', 'Presidente/Vice Presidente'),
    ('vice presidente', 'Presidente/Vice Presidente'),
    ('president', 'Presidente/Vice Presidente'),
    ('vp', 'Presidente/Vice Presidente'),
    ('vice-president', 'Presidente/Vice Presidente'),
    ('vice president', 'Presidente/Vice Presidente'),
    ('Founder', 'Founder/Co-Founder'),
    ('fundador', 'Founder/Co-Founder'),
    ('co-founder', 'Founder/Co-Founder'),
    ('cofounder', 'Founder/Co-Founder'),
    ('cofundador', 'Founder/Co-Founder'),
        ('diretor de analytics', 'Diretor de Analytics'),
    ('analytics director', 'Diretor de Analytics'),
    ('director of analytics', 'Diretor de Analytics'),
    ('diretor de tecnologia', 'Diretor de Tecnologia'),
    ('director of technology', 'Diretor de Tecnologia'),
    ('technology director', 'Diretor de Tecnologia'),
    ('diretor administrativo', 'Diretor Administrativo'),
    ('diretor de operações', 'Diretor de Operações'),
    ('director of operations', 'Diretor de Operações'),
    ('operations director', 'Diretor de Operações'),
    ('diretor financeiro', 'Diretor Financeiro'),
    ('financial director', 'Diretor Financeiro'),
    ('finance director', 'Diretor Financeiro'),
    ('diretor de marketing', 'Diretor de Marketing'),
    ('marketing director', 'Diretor de Marketing'),
    ('director of marketing', 'Diretor de Marketing'),
    ('diretor de logística/supply chain', 'Diretor de Logística/Supply Chain'),
    ('logistics director', 'Diretor de Logística/Supply Chain'),
    ('supply chain director', 'Diretor de Logística/Supply Chain'),
    ('director of logistics/supply chain', 'Diretor de Logística/Supply Chain'),
    ('diretor comercial', 'Diretor Comercial'),
    ('commercial director', 'Diretor Comercial'),
    ('director of commercial', 'Diretor Comercial'),
    ('diretor de inovação', 'Diretor de Inovação'),
    ('innovation director', 'Diretor de Inovação'),
    ('director of innovation', 'Diretor de Inovação'),
    ('diretor de planejamento', 'Diretor de Planejamento'),
    ('planning director', 'Diretor de Planejamento'),
    ('director of planning', 'Diretor de Planejamento'),
    ('diretor de mercado', 'Diretor de Mercado'),
    ('market director', 'Diretor de Mercado'),
    ('director of market', 'Diretor de Mercado'),
    ('diretor de negócios', 'Diretor de Negócios'),
    ('business director', 'Diretor de Negócios'),
    ('director of business', 'Diretor de Negócios'),
    ('diretor de processos', 'Diretor de Processos'),
    ('processes director', 'Diretor de Processos'),
    ('director of processes', 'Diretor de Processos'),
    ('diretor de produto', 'Diretor de Produto'),
    ('product director', 'Diretor de Produto'),
    ('director of product', 'Diretor de Produto'),
    ('diretor executivo', 'Diretor Executivo'),
    ('executive director', 'Diretor Executivo'),
    ('director of executive', 'Diretor Executivo'),
    ('diretor', 'Diretor'),
    ('diretora', 'Diretor'),
    ('director', 'Diretor'), 
    ('head of data', 'Head of Data'),
    ('head de dados', 'Head of Data'),
    ('data head', 'Head of Data'),
    ('head of technology', 'Head de Tecnologia'),
    ('head de tecnologia', 'Head de Tecnologia'),
    ('technology head', 'Head de Tecnologia'),
    ('head de inteligência de mercado', 'Head de Inteligência de Mercado'),
    ('head de inteligência de negocios', 'Head de Inteligência de Mercado'),
    ('head de BI', 'Head de Inteligência de Mercado'),
    ('head of BI', 'Head de Inteligência de Mercado'),
    ('head of market intelligence', 'Head de Inteligência de Mercado'),
    ('head of business intelligence', 'Head de Inteligência de Mercado'),
    ('market intelligence head', 'Head de Inteligência de Mercado'),
    ('head of business', 'Head of Business'),
    ('head de business', 'Head of Business'),
    ('business head', 'Head of Business'),
    ('head de operações', 'Head de Operações'),
    ('head of operations', 'Head de Operações'),
    ('operations head', 'Head de Operações'),
    ('head of growth', 'Head of Growth'),
    ('head de crescimento', 'Head of Growth'),
    ('growth head', 'Head of Growth'),
    ('head of logistics/supply chain', 'Head of Logistics/Supply Chain'),
    ('head de logística/cadeia de suprimentos', 'Head of Logistics/Supply Chain'),
    ('logistics head', 'Head of Logistics/Supply Chain'),
    ('supply chain head', 'Head of Logistics/Supply Chain'),
    ('head of marketing', 'Head of Marketing'),
    ('head de marketing', 'Head of Marketing'),
    ('marketing head', 'Head of Marketing'),
    ('head of finance', 'Head of Finance'),
    ('head de finanças', 'Head of Finance'),
    ('finance head', 'Head of Finance'),
    ('head de estratégia', 'Head de Estratégia'),
    ('head of strategy', 'Head de Estratégia'),
    ('strategy head', 'Head de Estratégia'),
    ('head de planejamento', 'Head de Planejamento'),
    ('head of planning', 'Head de Planejamento'),
    ('planning head', 'Head de Planejamento'),
    ('head de produto', 'Head de Produto'),
    ('head of product', 'Head de Produto'),
    ('product head', 'Head de Produto'),
    ('head of fp&a', 'Head of FP&A'),
    ('head of financial planning and analysis', 'Head of FP&A'),
    ('fp&a head', 'Head of FP&A'),
    ('head of innovation', 'Head of Innovation'),
    ('head de inovação', 'Head of Innovation'),
    ('innovation head', 'Head of Innovation'),
    ('head', 'Head'),
    ('superintendente', 'Superintendente'),
    ('gerente de analytics', 'Gerente de Analytics'),
    ('analytics manager', 'Gerente de Analytics'),
    ('manager of analytics', 'Gerente de Analytics'),
    ('gerente de tecnologia', 'Gerente de Tecnologia'),
    ('technology manager', 'Gerente de Tecnologia'),
    ('manager of technology', 'Gerente de Tecnologia'),
    ('gerente de inteligencia de mercado', 'Gerente de Inteligência de Mercado'),
    ('gerente de business intelligence', 'Gerente de Inteligência de Mercado'),
    ('gerente de inteligencia', 'Gerente de Inteligência de Mercado'),
    ('gerente de BI', 'Gerente de Inteligência de Mercado'),
    ('business intelligence manager', 'Gerente de Inteligência de Mercado'),
    ('BI manager', 'Gerente de Inteligência de Mercado'),
    ('intelligence manager', 'Gerente de Inteligência de Mercado'),
    ('manager of business intelligence', 'Gerente de Inteligência de Mercado'),
    ('gerente de operações', 'Gerente de Operações'),
    ('operations manager', 'Gerente de Operações'),
    ('manager of operations', 'Gerente de Operações'),
    ('gerente de logística/supply chain', 'Gerente de Logística/Supply Chain'),
    ('logistics manager', 'Gerente de Logística/Supply Chain'),
    ('supply chain manager', 'Gerente de Logística/Supply Chain'),
    ('manager of logistics/supply chain', 'Gerente de Logística/Supply Chain'),
    ('gerente de desenvolvimento', 'Gerente de Desenvolvimento'),
    ('development manager', 'Gerente de Desenvolvimento'),
    ('manager of development', 'Gerente de Desenvolvimento'),
    ('gerente de estratégia', 'Gerente de Estratégia'),
    ('strategy manager', 'Gerente de Estratégia'),
    ('manager of strategy', 'Gerente de Estratégia'),
    ('gerente de facilities', 'Gerente de Facilities'),
    ('facilities manager', 'Gerente de Facilities'),
    ('manager of facilities', 'Gerente de Facilities'),
    ('gerente de inovação', 'Gerente de Inovação'),
    ('innovation manager', 'Gerente de Inovação'),
    ('manager of innovation', 'Gerente de Inovação'),
    ('gerente de marketing', 'Gerente de Marketing'),
    ('marketing manager', 'Gerente de Marketing'),
    ('manager of marketing', 'Gerente de Marketing'),
    ('gerente de negócios', 'Gerente de Negócios'),
    ('business manager', 'Gerente de Negócios'),
    ('manager of business', 'Gerente de Negócios'),
    ('gerente de planejamento', 'Gerente de Planejamento'),
    ('planning manager', 'Gerente de Planejamento'),
    ('manager of planning', 'Gerente de Planejamento'),
    ('gerente de produto', 'Gerente de Produto'),
    ('product manager', 'Gerente de Produto'),
    ('manager of product', 'Gerente de Produto'),
    ('manager', 'Gerente'), 
    ('gerente', 'Gerente'),
    ('Project Management Officer', 'PMO'),
    ('Project Management', 'PMO')
]

In [55]:
def replace_job_titles(title):
    for pattern, replacement in job_title_replacements:
        if re.search(pattern, title, re.IGNORECASE):
            return replacement
    return title

In [11]:
treino['Cargo'] = treino['Cargo'].fillna('').apply(replace_job_titles)
teste['Cargo'] = teste['Cargo'].fillna('').apply(replace_job_titles)

In [ ]:
features = ['Semana', 'Empresa', 'Setor do LinkedIn', 'Número de Funcionários', 'Cargo', 'Portfólio']
target = 'STATUS'

X_train = treino[features]
y_train = treino[target]

X_test = teste[features]

# Fill missing values in both datasets
X_train.fillna('missing', inplace=True)
X_test.fillna('missing', inplace=True)

# Convert 'Número de Funcionários' to numeric, handle errors by coercing to NaN
X_train['Número de Funcionários'] = pd.to_numeric(X_train['Número de Funcionários'], errors='coerce')
X_test['Número de Funcionários'] = pd.to_numeric(X_test['Número de Funcionários'], errors='coerce')

# Fill missing numeric values with the mean from the training data
mean_num_func = X_train['Número de Funcionários'].mean()
X_train['Número de Funcionários'].fillna(mean_num_func, inplace=True)
X_test['Número de Funcionários'].fillna(mean_num_func, inplace=True)

# Specify categorical columns
cat_cols = ['Semana', 'Empresa', 'Setor do LinkedIn', 'Cargo', 'Portfólio']

# Initialize the CatBoostClassifier model
model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    loss_function='MultiClass',
    verbose=100
)

# Train the model
model.fit(X_train, y_train, cat_features=cat_cols)

# Make predictions on the test data
predictions = model.predict(X_test)

In [ ]:
# Extract the predicted class labels
predicted_labels = [pred[0] for pred in predictions]

# Add predictions to the test dataframe
teste['Predicted_STATUS'] = predicted_labels

# Save the predictions to a CSV file
teste.to_csv('teste_predictions.csv', index=False)